In [1]:
from casadi import *

# Paper Cascaded Nonlinear MPC for Realtime Quadrotor Position Tracking. Schlagenhauf

# Define system, static/kinematic/dynamic model. Typically Differential (Algebraic) equations

# Expression graphs represent a computation in the computer memory. SX, MX


Ix = 0.0000166  #Moment of inertia around p_WB_W_x-axis, source: Julian Förster's ETH Bachelor Thesis
Iy = 0.0000167  #Moment of inertia around p_WB_W_y-axis, source: Julian Förster's ETH Bachelor Thesis
Iz = 0.00000293 #Moment of inertia around p_WB_W_z-axis, source: Julian Förster's ETH Bachelor Thesis
m = 0.029 #mass of Crazyflie 2.1
g = 9.81

Nx = 12
Nu = 4
Nhoriz = 10

# x(t) = [x, y, z, phi, theta, psi, x_dot, y_dot, z_dot, phi_dot, ˙theta_dot, psi_dot]T
X = MX.sym("x",Nx)
U = MX.sym("u",Nu)


x_pos = X[0]  # x-position
y = X[1]  # y-position
z = X[2]  # z-position
phi = X[3]  # phi-angle, Euler angles
theta = X[4]  # theta-angle, Euler angles
psi = X[5]  # psi-angle, Euler angles
x_pos_dot = X[6]  # x velocity
y_dot = X[7]  # y velocity
z_dot = X[8]  # z velocity
phi_dot = X[9]  # phi_dot, angular velocity
theta_dot = X[10]  # theta_dot
psi_dot = X[11]  # psi-dot

thrust = U[0]
tau_phi = U[1]
tau_theta = U[2]
tau_psi = U[3]


# x˙(t) = [x_dot, y_dot, z_dot, phi_dot, ˙theta_dot, psi_dot, x_ddot, y_ddot, z_ddot, phi_ddot, ˙theta_ddot, psi_ddot]T


x_pos_ddot = (cos(phi) * sin(phi) * cos(psi) + sin(phi) * sin(phi))  * thrust / m
y_ddot = (cos(phi) * sin(phi) * cos(psi) - sin(phi) * sin(phi))  * thrust / m
z_ddot = -g + (cos(phi) * cos(theta)) * thrust / m
phi_ddot = theta_dot * psi_dot * (Iy - Iz) / (Ix) + tau_phi / Ix
theta_ddot = phi_dot * psi_dot * (Iz - Ix) / (Iy) + tau_phi / Iy
psi_ddot = theta_dot * phi_dot * (Ix - Iy) / (Iz) + tau_phi / Iz

x_dot = vertcat(x_pos_dot, y_dot, z_dot, phi_dot, theta_dot, psi_dot, x_pos_ddot, y_ddot, z_ddot, phi_ddot, theta_ddot, psi_ddot)

f = Function('f', [X, U], [x_dot], ['x','u'],['x_dot'])

f

Function(f:(x[12],u[4])->(x_dot[12]) MXFunction)

In [4]:
# Define time horizon and control intervals
T = 10  # Total time horizon
N = 20  # Number of control intervals
dt = T / N  # Duration of each control interval

# Corrected definition of the integrator
# Note: 't0' is not explicitly needed if the integration starts at time 0
ode = {'x': X, 'p': U, 'ode': f(X, U)}
integrator = casadi.integrator('integrator', 'cvodes', ode, 0, dt)

ode

{'x': MX(x),
 'p': MX(u),
 'ode': MX(@1=x[9], @2=x[10], @3=x[11], @4=x[3], @5=x[5], @6=u[0], @7=u[1], vertcat(x[6], x[7], x[8], @1, @2, @3, (((((cos(@4)*sin(@4))*cos(@5))+sq(sin(@4)))*@6)/0.029), (((((cos(@4)*sin(@4))*cos(@5))-sq(sin(@4)))*@6)/0.029), (-9.81+(((cos(@4)*cos(x[4]))*@6)/0.029)), (((1.377e-05*(@2*@3))/1.66e-05)+(@7/1.66e-05)), (((-1.367e-05*(@1*@3))/1.67e-05)+(@7/1.67e-05)), (((-1e-07*(@2*@1))/2.93e-06)+(@7/2.93e-06))))}

In [5]:
integrator

Function(integrator:(x0[12],z0[0],p[4],u[0],adj_xf[],adj_zf[],adj_qf[])->(xf[12],zf[0],qf[0],adj_x0[],adj_z0[],adj_p[],adj_u[]) CvodesInterface)

In [6]:
# Example desired state (you should define this based on your specific goal)
x_des = MX.sym('x_des', Nx)
u_des = MX.zeros(Nu)  # Assuming desired control is to minimize input usage

# Quadratic cost function components
Q = MX.sym('Q', Nx, Nx)  # State weighting matrix
R = MX.sym('R', Nu, Nu)  # Control input weighting matrix

# Initialize cost
cost = 0

# Sum over the prediction horizon
for k in range(N):
    # Predicted state at step k
    Xk = integrator(x0=X[:, k], p=U[:, k])['xf']
    
    # Update cost function
    cost += (Xk - x_des).T @ Q @ (Xk - x_des) + (U[:, k] - u_des).T @ R @ (U[:, k] - u_des)


RuntimeError: .../casadi/core/sparsity_internal.cpp:2196: Assertion "in_range(cc, -size2()+ind1, size2()+ind1)" failed:
Out of bounds error. Got elements in range [1,1], which is outside the range [-1,1).